<a href="https://colab.research.google.com/github/dkabulski/StravaDownloader/blob/main/strava_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONNECT GOOGLE DRIVE AND SAVE DATAFRAME & RESPONSES

In [ ]:
import pandas as pd
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Mount Google Drive. Once mounted will work as mounted Data Lake / local dir
# can also run drive.flush_and_unmount()
# if it fails just restart the runtime
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

#%ls
#%mkdir strava_outputs

#https://www.youtube.com/watch?v=sgscChKfGyg&t=627s

In [ ]:
# go to strava_outputs directory where we'll be saving all results in raw .json format
%cd /content/drive/My\ Drive/python_workbooks/strava_outputs


# KEYS READ

In [ ]:
# move this to secrets store
STRAVA_CLIENT_ID = intToGetfromStrava
STRAVA_CLIENT_SECRET = 'updatefromStravaAPIPage' # keep secret! static
STRAVA_ACCESS_TOKEN = 'AAA' #this one expires - will be updated using refresh_token
STRAVA_REFRESH_TOKEN = 'STRAVAPROVIDES' #this can be used to refresh the above, static. keep secret!


# STRAVA ACTIVITY READ classic manual method

In [ ]:
# connect to google drive spreadsheet for later to load the activities
import gspread
from oauth2client.service_account import ServiceAccountCredentials

%cd /content/drive/My\ Drive/python_workbooks/

# use creds to create a client to interact with the Google Drive API
#scope = ['https://spreadsheets.google.com/feeds']
scope = ['https://spreadsheets.google.com/feeds' + ' ' +'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
gc = gspread.authorize(credentials)



In [ ]:

import requests
import json
import urllib
from requests_oauthlib import OAuth2Session
base_url = "https://www.strava.com/oauth/authorize?"
known_response = requests.get(base_url)
redirect_uri = "http://localhost/exchange_token"


#first scope is for all (including private), second only public activities
strava_scope = 'read,read_all,activity:read,activity:read_all,profile:read_all'
#strava_scope = 'read,read_all,activity:read'
oauth2_session = OAuth2Session(STRAVA_CLIENT_ID,  redirect_uri=redirect_uri, scope=strava_scope)
authorization_request, state = oauth2_session.authorization_url(base_url)#,STATE)
print("Click on the following link to present the user with sign in form where they authenticate and approve access to your application.")
print(authorization_request) 

In [ ]:
code = 'pasteCodeHere' #update using strava OAuth2Session above. Copy the code bit.

In [ ]:
#GETTING NEW ACCESS TOKEN USING REFRESH_TOKEN METHOD
import requests
from datetime import datetime

import urllib3
urllib3.disable_warnings()


auth_url ="https://www.strava.com/oauth/token"
payload = {
    'client_id' : STRAVA_CLIENT_ID,
    'client_secret' : STRAVA_CLIENT_SECRET,
    #'refresh_token' : STRAVA_REFRESH_TOKEN,
    'code': code,
    #'grant_type' : "refresh_token",
    'grant_type': 'authorization_code'
    #'scope': 'read,read_all,activity:read,activity:read_all,profile:read_all',
    #'f':'json'
}
print("Requesting the token...\n")
res = requests.post(auth_url,data=payload,verify=False)
print(res.json())

access_token = res.json()['access_token']
expiry_ts = res.json()['expires_at']
print("New token will expire at: ",end='\t')
print(datetime.utcfromtimestamp(expiry_ts).strftime('%Y-%m-%d %H:%M:%S'))
print(access_token)


############
activites_url = "https://www.strava.com/api/v3/athlete/activities"
activites_url = "https://www.strava.com/api/v3/athlete"

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}

my_dataset = requests.get(activites_url, headers=header, params=param).json()
print("Authenticated Athleete: ",my_dataset['username'])

In [ ]:
#READ ATHLETE DETAILS - activity count
import requests
import json

athlete_id = yourAthleteID
url = 'https://www.strava.com/api/v3/athletes/'+str(athlete_id)+'/stats'
print(url)


output_list = []
params = {'access_token': access_token, 'per_page': 100, 'page': 1}


r = requests.get(url, params = params)
print("Reading page ","finished with ",str(r)," code")
data = r.json()
print(data)
#activities_count = data['all_run_totals']['count'] + data['all_swim_totals']['count'] + data['all_ride_totals']['count']
#activities_count =data['all_ride_totals']['count']
#print("Total run, ride, swim activities: ",activities_count)

In [ ]:
# #READ ATHLETE's SEGMENTS
# import requests
# import json

# athlete_id = athleteID
# url = 'https://www.strava.com/api/v3/segments/starred'
# print(url)


# output_list = []
# params = {'access_token': access_token, 'per_page': 100, 'page': 1}


# r = requests.get(url, params = params)
# print("Reading page ","finished with ",str(r)," code")
# data = r.json()

# for segment in data:
#   print(segment)

In [ ]:

# #READ Segment details
# import requests
# import json

# segment_id = 6691038 #swains lane
# url = 'https://www.strava.com/api/v3/segments/'+str(segment_id)
# print(url)


# output_list = []
# params = {'access_token': access_token, 'per_page': 100, 'page': 1}


# r = requests.get(url, params = params)
# print("Reading segment",segment_id,"finished with ",str(r)," code")
# data = r.json()

# #print(data)


In [ ]:
# segment_details = []
# for k,v in data.items():
#   segdict = {}
#   segdict[k] = v

#   segment_details.append(segdict)
# segment_details

In [ ]:
# go to strava_outputs directory where we'll be saving all results in raw .json format
%cd /content/drive/My\ Drive/python_workbooks/strava_outputs


In [ ]:
import requests
import json

url = 'https://www.strava.com/api/v3/athlete/activities'

#with 100 activities per page you need to calculate how many requests to make. Divide activities you've made by 100
#in this case it's good to load all actities. Doesn't take long and those can historically change (kudos, flagged, updated shoes etc.)

page = 1
#starting_page = 1
#limit = 42

output_list = []
params = {'access_token': access_token, 'per_page': 100, 'page': 1}
#for i in range(starting_page, limit):
content = 1
while content > 0:
  params['page'] = page

  r = requests.get(url, params = params)
  print("Reading page",params['page'],"finished with",str(r)," code")
  data = r.json()
  content = len(data)
  page = page+1





  # save outputs to drive as json file
  filename = "activities_"+str(params['page'])+"_25"+".json"
  with open(filename, 'w') as f:
    f.write(json.dumps(data))

  output_list.append(data)

In [ ]:
print(len(output_list))
#output_list[0]

In [ ]:
available_fields = []
for item in output_list[0][0]:
  available_fields.append(item)
print(available_fields)

In [ ]:
# TODO
# - tag activities with incorrect HR as heartrate_opt_out
# - use push to update activities with a correct gear based on what's in Excel
# - tidy up the code & perhaps authentication? (so it's just a pop-up link, without having a need to copy 'code')

In [ ]:
actlist = []
for page in output_list:
  for item in page:
    actdict = {}
    for field in available_fields:
      if field in item:
        actdict[field] = item[field]
    actlist.append(actdict)

actDF = pd.DataFrame(actlist)
print(actDF.shape)

actDF['date_datetime'] = pd.to_datetime(actDF['start_date_local'],utc=True)
actDF['date_date'] =actDF['date_datetime'].dt.date
actDF['hour'] = actDF['date_datetime'].dt.hour
actDF['month'] = actDF['date_datetime'].dt.month
actDF['year'] = actDF['date_datetime'].dt.year
actDF['dateActType'] = actDF['date_date'].astype(str) + actDF['type']
#actDF['stringMatch'] = actDF['date_date']+actDF['type']

cols = actDF.columns.tolist()
cols = cols[-1:] + cols[:-1]
actDF = actDF[cols]

actDF.head(3)

In [ ]:
# filter activities by id:
# actDF[actDF['upload_id']==4768757768]

In [1]:

# actDF[actDF['heartrate_opt_out']==True]

In [ ]:
actDF

# Load data into googlesheet
https://console.developers.google.com/?pli=1

In [ ]:
#%pip install df2gspread

import gspread
from oauth2client.service_account import ServiceAccountCredentials

%cd /content/drive/My\ Drive/python_workbooks

# use creds to create a client to interact with the Google Drive API
#scope = ['https://spreadsheets.google.com/feeds']
scope = ['https://spreadsheets.google.com/feeds' + ' ' +'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
gc = gspread.authorize(credentials)


# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
#sheet = gc.open("daily run results").sheet


In [ ]:
spreadsheet = gc.open("daily run results")
#this assumes that first sheet is the STRAVA_TEST sheet, if it doesn't exist it will create it
try:
  worksheet = spreadsheet.add_worksheet(title="STRAVA_TEST", rows="10000", cols="20")
except:
  worksheet = spreadsheet.get_worksheet(0)

In [ ]:

import gspread
from gspread_dataframe import set_with_dataframe

# APPEND DATA TO SHEET
#sheet = select_worksheet('worksheet key here',1)
set_with_dataframe(worksheet, actDF) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET